In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import digits
import os 
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import random
import datetime
from tensorflow import keras


In [2]:
os.listdir('data/')

['Mouth2901.jpg',
 'Ear3358.jpg',
 'Ear3359.jpg',
 'Ear3360.jpg',
 'Ear3361.jpg',
 'Ear3362.jpg',
 'Ear3363.jpg',
 'Ear3364.jpg',
 'Ear3365.jpg',
 'Ear3366.jpg',
 'Ear3367.jpg',
 'Ear3368.jpg',
 'Ear1.jpg',
 'Ear2.jpg',
 'Ear3.jpg',
 'Ear4.jpg',
 'Ear5.jpg',
 'Ear6.jpg',
 'Ear7.jpg',
 'Ear8.jpg',
 'Ear9.jpg',
 'Ear10.jpg',
 'Ear11.jpg',
 'Ear12.jpg',
 'Ear13.jpg',
 'Ear14.jpg',
 'Ear15.jpg',
 'Ear16.jpg',
 'Ear17.jpg',
 'Ear18.jpg',
 'Ear19.jpg',
 'Ear20.jpg',
 'Ear21.jpg',
 'Ear22.jpg',
 'Ear23.jpg',
 'Ear24.jpg',
 'Ear25.jpg',
 'Ear26.jpg',
 'Ear27.jpg',
 'Ear28.jpg',
 'Ear29.jpg',
 'Ear30.jpg',
 'Ear31.jpg',
 'Ear32.jpg',
 'Ear33.jpg',
 'Ear34.jpg',
 'Ear35.jpg',
 'Ear36.jpg',
 'Ear37.jpg',
 'Ear38.jpg',
 'Ear39.jpg',
 'Ear40.jpg',
 'Ear41.jpg',
 'Ear42.jpg',
 'Ear43.jpg',
 'Ear44.jpg',
 'Ear45.jpg',
 'Ear46.jpg',
 'Ear47.jpg',
 'Ear48.jpg',
 'Ear49.jpg',
 'Ear50.jpg',
 'Ear51.jpg',
 'Ear52.jpg',
 'Ear53.jpg',
 'Ear54.jpg',
 'Ear55.jpg',
 'Ear56.jpg',
 'Ear57.jpg',
 'Ear58.jpg',
 'E

In [2]:
label_dict = {'Ear':0, 'Eye':1, 'Hand':2, 'Mouth':3, 'Nose':4}

inverse_dict = {v: k for k, v in label_dict.items()}

remove_digits = str.maketrans('', '', digits)

In [3]:
example_image = 'data\\Mouth1.jpg'

example_image.split('.')[0].split('\\')[1].translate(remove_digits)

'Mouth'

In [4]:
def preprocess_images(img):

    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (128,128))

    label_temp = img.split('.')[0].split('\\')[1].translate(remove_digits)
    label = label_dict[label_temp]
    return resized/255, label

In [6]:
preprocess_images('data\\Mouth255.jpg')[0]

array([[0.76862745, 0.76470588, 0.76470588, ..., 0.12941176, 0.1372549 ,
        0.1372549 ],
       [0.76470588, 0.76470588, 0.76470588, ..., 0.12941176, 0.13333333,
        0.13333333],
       [0.76470588, 0.76078431, 0.75686275, ..., 0.12941176, 0.13333333,
        0.12941176],
       ...,
       [0.57254902, 0.58431373, 0.58039216, ..., 0.19607843, 0.24313725,
        0.2745098 ],
       [0.57254902, 0.58431373, 0.58039216, ..., 0.25882353, 0.29803922,
        0.30588235],
       [0.57254902, 0.58039216, 0.58039216, ..., 0.31764706, 0.3372549 ,
        0.3254902 ]])

In [5]:
def load_images():
    images, labels = [], []

    for i in os.listdir('data'):
        processed_img, label_img = preprocess_images(os.path.join('data', i))


        images.append(processed_img)
        labels.append(label_img)
    return np.array(images), np.array(labels)    

In [6]:
image_data, label_data = load_images()

In [7]:
image_data.shape

(16561, 128, 128)

In [68]:
random.random()

0.1472777635096475

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
def create_partition(img, lab):
    x_train, y_train, x_val, y_val = [], [], [], [],

    for i in zip(img, lab):
        rand = random.random()
        if rand > 0.1:
            x_train.append(i[0])
            y_train.append(i[1])
        else:
            x_val.append(i[0])
            y_val.append(i[1])
        
    assert len(x_train) == len(y_train)

    return np.array(x_train), np.array(y_train), np.array(x_val), np.array(y_val)             

In [10]:
x_train, y_train, x_val, y_val = create_partition(image_data, label_data)

In [11]:
x_train.shape

(14943, 128, 128)

In [15]:
def build_model():

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(16, (3,3),padding='same', activation='relu', input_shape=(128,128,1)))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))
    model.add(tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.Conv2D(64, (3,3),padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.Conv2D(128, (3,3),padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    return model

In [16]:
model = build_model()
name = datetime.datetime.now().strftime('%H-%M-%S')

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 128, 16)      160       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 127, 127, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 64)      

In [18]:
stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                        patience = 5,
                                        mode = 'min',
                                        restore_best_weights=True)

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'log/{name}')                                        

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss='sparse_categorical_crossentropy',
              metrics= ['accuracy'])

In [22]:
x_train = x_train.reshape(x_train.shape[0], 128,128,1)
x_val = x_val.reshape(x_val.shape[0], 128,128,1)
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)

In [23]:
history = model.fit(x=x_train, y=y_train,batch_size=128, validation_data=(x_val,y_val), epochs=10, callbacks=[stop, tensorboard])

Epoch 1/10
117/117 [==============================] - 17s 92ms/step - loss: 1.5980 - accuracy: 0.2129 - val_loss: 1.5863 - val_accuracy: 0.2250
Epoch 2/10
117/117 [==============================] - 10s 83ms/step - loss: 1.5777 - accuracy: 0.2417 - val_loss: 1.5637 - val_accuracy: 0.3096
Epoch 3/10
117/117 [==============================] - 10s 83ms/step - loss: 1.5533 - accuracy: 0.3455 - val_loss: 1.5342 - val_accuracy: 0.3733
Epoch 4/10
117/117 [==============================] - 10s 83ms/step - loss: 1.5204 - accuracy: 0.3985 - val_loss: 1.4965 - val_accuracy: 0.4258
Epoch 5/10
117/117 [==============================] - 10s 83ms/step - loss: 1.4815 - accuracy: 0.4324 - val_loss: 1.4535 - val_accuracy: 0.4543
Epoch 6/10
117/117 [==============================] - 10s 83ms/step - loss: 1.4362 - accuracy: 0.4873 - val_loss: 1.4041 - val_accuracy: 0.5402
Epoch 7/10
117/117 [==============================] - 9s 80ms/step - loss: 1.3847 - accuracy: 0.5582 - val_loss: 1.3487 - val_accuracy: 

In [ ]:
model.predict_generator

In [ ]:
model_json=model.to_json()
with open("model.json",'w') as file:
    file.write(model_json)
model.save_weights("model.h5")    

In [ ]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

load_model = model_from_json(loaded_model_json)
load_model.load_weights("model.h5")

In [ ]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the model
model = load_model('model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image
image = Image.open('eye_1.jpg')
#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)